In [1]:
import os
import numpy as np
from glob import glob
import pandas as pd
import matplotlib.pyplot as plt
import cv2 as cv
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose
from tensorflow.keras import Sequential
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.saving import load_model
from skimage.morphology import disk

In [2]:
ae = load_model("autoencoder_seg.keras")
ae.summary()
threshold = 252



Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 384, 512, 1)]     0         
                                                                 
 conv2d (Conv2D)             (None, 192, 256, 16)      160       
                                                                 
 conv2d_1 (Conv2D)           (None, 96, 128, 8)        1160      
                                                                 
 conv2d_2 (Conv2D)           (None, 48, 64, 4)         292       
                                                                 
 conv2d_transpose (Conv2DTr  (None, 96, 128, 4)        148       
 anspose)                                                        
                                                                 
 conv2d_transpose_1 (Conv2D  (None, 192, 256, 8)       296       
 Transpose)                                                

In [3]:
train_ims = sorted(glob(os.path.join('.\Train\Train','*.jpg')))
for i in range(len(train_ims)):
    train_ims[i] = train_ims[i].replace('.\\Train\\Train\\','./Train/Modified/')
print(len(train_ims))
print(train_ims[:5])

seg_ims = sorted(glob(os.path.join('.\Train\Train','*.png')))
print(len(seg_ims))

for i in range(len(seg_ims)):
    seg_ims[i] = seg_ims[i].replace('_seg.png','.jpg')
    seg_ims[i] = seg_ims[i].replace('.\\Train\\Train\\','./Train/Modified/')
print(seg_ims[:5])

undone_train = np.array([x for x in train_ims if x not in seg_ims])
print(undone_train[:5])


test_ims = sorted(glob(os.path.join('.\Test\Test','*.jpg')))
for i in range(len(test_ims)):
    test_ims[i] = test_ims[i].replace('.\\Test\\Test\\','./Test/Modified/')
print(len(test_ims))
print(test_ims[:5])

segtest_ims = sorted(glob(os.path.join('.\Test\Test','*.png')))
print(len(segtest_ims))

for i in range(len(segtest_ims)):
    segtest_ims[i] = segtest_ims[i].replace('_seg.png','.jpg')
    segtest_ims[i] = segtest_ims[i].replace('.\\Test\\Test\\','./Test/Modified/')
print(segtest_ims[:5])

undone_test = np.array([x for x in test_ims if x not in segtest_ims])
print(undone_test[:5])

18998
['./Train/Modified/ISIC_0000000.jpg', './Train/Modified/ISIC_0000001.jpg', './Train/Modified/ISIC_0000002.jpg', './Train/Modified/ISIC_0000003.jpg', './Train/Modified/ISIC_0000004.jpg']
1945
['./Train/Modified/ISIC_0000000.jpg', './Train/Modified/ISIC_0000001.jpg', './Train/Modified/ISIC_0000003.jpg', './Train/Modified/ISIC_0000004.jpg', './Train/Modified/ISIC_0000007.jpg']
['./Train/Modified/ISIC_0000002.jpg' './Train/Modified/ISIC_0000010.jpg'
 './Train/Modified/ISIC_0000114.jpg' './Train/Modified/ISIC_0000118.jpg'
 './Train/Modified/ISIC_0000174.jpg']
6333
['./Test/Modified/ISIC_0000006.jpg', './Test/Modified/ISIC_0000011.jpg', './Test/Modified/ISIC_0000014.jpg', './Test/Modified/ISIC_0000018.jpg', './Test/Modified/ISIC_0000022.jpg']
648
['./Test/Modified/ISIC_0000006.jpg', './Test/Modified/ISIC_0000011.jpg', './Test/Modified/ISIC_0000014.jpg', './Test/Modified/ISIC_0000018.jpg', './Test/Modified/ISIC_0000022.jpg']
['./Test/Modified/ISIC_0000130.jpg' './Test/Modified/ISIC_0000

In [4]:
filenametrain = []
for i in range(len(undone_train)):
    name = undone_train[i]
    name = name.replace('.\\Train\\Train\\','./Train/Modified/')
    name = name.replace('.jpg','_seg.png')
    filenametrain.append(name)
print(filenametrain[:5])

filenametest = []
for i in range(len(undone_test)):
    name = undone_test[i]
    name = name.replace('.\\Test\\Test\\','./Test/Modified/')
    name = name.replace('.jpg','_seg.png')
    filenametest.append(name)
print(filenametest[:5])

['./Train/Modified/ISIC_0000002_seg.png', './Train/Modified/ISIC_0000010_seg.png', './Train/Modified/ISIC_0000114_seg.png', './Train/Modified/ISIC_0000118_seg.png', './Train/Modified/ISIC_0000174_seg.png']
['./Test/Modified/ISIC_0000130_seg.png', './Test/Modified/ISIC_0000195_seg.png', './Test/Modified/ISIC_0000551_seg.png', './Test/Modified/ISIC_0003051_seg.png', './Test/Modified/ISIC_0004166_seg.png']


In [5]:
m = np.concatenate((np.zeros((384,64)),np.concatenate((np.ones((384,384)),np.zeros((384,64))),axis=1)),axis=1)

In [6]:
for i in range(0,len(undone_train),2):
    if i%100==0:
        print(i)
    z = np.array([1/255*np.reshape(cv.resize(cv.imread(x,cv.IMREAD_GRAYSCALE),(512,384)),(384,512,1)) for x in undone_train[i:i+2]])
    p = ae(z).numpy()
    for k in range(2):
        mi = np.min(p[k])
        ma = np.max(p[k])
        im = 255/(ma-mi)*(p[k]-mi)
        thresh = (im>threshold).astype(np.uint8)

        s0 = np.sum(thresh,axis=0)
        s1 = np.sum(s0[:128])
        s2 = np.sum(s0[128:256])
        s3 = np.sum(s0[256:384])
        if s1>s2 or s3>s2:
            thresh = (thresh.reshape(384,512)*m).astype(np.uint8)
        
        thresh = cv.morphologyEx(thresh, cv.MORPH_CLOSE, cv.getStructuringElement(cv.MORPH_ELLIPSE, (60, 45)))
        thresh = np.pad(255*thresh,((1,1),(1,1)),'constant',constant_values=(0,0))
        # Find largest contour in intermediate image
        cnts, _ = cv.findContours(thresh, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_NONE)
        if len(cnts)!=0:
            cnt = max(cnts, key=cv.contourArea)
            # Output
            out = np.zeros(thresh.shape, np.uint8)
            cv.drawContours(out, [cnt], -1, 255,cv.FILLED)
         
            cv.imwrite(filenametrain[i+k],out[1:385,1:513])

10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000


IndexError: index 1 is out of bounds for axis 0 with size 1

In [6]:
for i in range(0,len(undone_test),2):
    if i%100==0:
        print(i)
    z = np.array([1/255*np.reshape(cv.resize(cv.imread(x,cv.IMREAD_GRAYSCALE),(512,384)),(384,512,1)) for x in undone_test[i:i+2]])
    p = ae(z).numpy()
    for k in range(2):
        mi = np.min(p[k])
        ma = np.max(p[k])
        im = 255/(ma-mi)*(p[k]-mi)
        thresh = (im>threshold).astype(np.uint8)

        s0 = np.sum(thresh,axis=0)
        s1 = np.sum(s0[:128])
        s2 = np.sum(s0[128:256])
        s3 = np.sum(s0[256:384])
        if s1>s2 or s3>s2:
            thresh = (thresh.reshape(384,512)*m).astype(np.uint8)
        
        thresh = cv.morphologyEx(thresh, cv.MORPH_CLOSE, cv.getStructuringElement(cv.MORPH_ELLIPSE, (60, 45)))
        thresh = np.pad(255*thresh,((1,1),(1,1)),'constant',constant_values=(0,0))
        # Find largest contour in intermediate image
        cnts, _ = cv.findContours(thresh, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_NONE)
        if len(cnts)!=0:
            cnt = max(cnts, key=cv.contourArea)
            # Output
            out = np.zeros(thresh.shape, np.uint8)
            cv.drawContours(out, [cnt], -1, 255,cv.FILLED)
         
            cv.imwrite(filenametest[i+k],out[1:385,1:513])

3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600


IndexError: index 1 is out of bounds for axis 0 with size 1

In [14]:
seg_ims = sorted(glob(os.path.join('.\Train\Train','*.png')))
print(len(seg_ims))
print(seg_ims[:5])
train_dest = []
for i in range(len(seg_ims)):
    name = seg_ims[i]
    name = name.replace('.\\Train\\Train\\','./Train/Modified/')
    train_dest.append(name)
print(train_dest[:5])

segtest_ims = sorted(glob(os.path.join('.\Test\Test','*.png')))
print(len(segtest_ims))
print(segtest_ims[:5])
test_dest = []
for i in range(len(segtest_ims)):
    name = segtest_ims[i]
    name = name.replace('.\\Test\\Test\\','./Test/Modified/')
    test_dest.append(name)
print(test_dest[:5])


1945
['.\\Train\\Train\\ISIC_0000000_seg.png', '.\\Train\\Train\\ISIC_0000001_seg.png', '.\\Train\\Train\\ISIC_0000003_seg.png', '.\\Train\\Train\\ISIC_0000004_seg.png', '.\\Train\\Train\\ISIC_0000007_seg.png']
['./Train/Modified/ISIC_0000000_seg.png', './Train/Modified/ISIC_0000001_seg.png', './Train/Modified/ISIC_0000003_seg.png', './Train/Modified/ISIC_0000004_seg.png', './Train/Modified/ISIC_0000007_seg.png']
648
['.\\Test\\Test\\ISIC_0000006_seg.png', '.\\Test\\Test\\ISIC_0000011_seg.png', '.\\Test\\Test\\ISIC_0000014_seg.png', '.\\Test\\Test\\ISIC_0000018_seg.png', '.\\Test\\Test\\ISIC_0000022_seg.png']
['./Test/Modified/ISIC_0000006_seg.png', './Test/Modified/ISIC_0000011_seg.png', './Test/Modified/ISIC_0000014_seg.png', './Test/Modified/ISIC_0000018_seg.png', './Test/Modified/ISIC_0000022_seg.png']


In [16]:
for i in range(len(seg_ims)):
    im = cv.resize(cv.imread(seg_ims[i],cv.IMREAD_GRAYSCALE),(512,384))
    cv.imwrite(train_dest[i],im)

In [17]:
for i in range(len(segtest_ims)):
    im = cv.resize(cv.imread(segtest_ims[i],cv.IMREAD_GRAYSCALE),(512,384))
    cv.imwrite(test_dest[i],im)

In [22]:
segtest_ims = sorted(glob(os.path.join('.\Test\Modified','*.png')))
print(len(segtest_ims))
print(segtest_ims[:5])
test_ims = sorted(glob(os.path.join('.\Test\Modified','*.jpg')))
print(len(test_ims))
print(test_ims[:5])

6308
['.\\Test\\Modified\\ISIC_0000006_seg.png', '.\\Test\\Modified\\ISIC_0000011_seg.png', '.\\Test\\Modified\\ISIC_0000014_seg.png', '.\\Test\\Modified\\ISIC_0000018_seg.png', '.\\Test\\Modified\\ISIC_0000022_seg.png']
6333
['.\\Test\\Modified\\ISIC_0000006.jpg', '.\\Test\\Modified\\ISIC_0000011.jpg', '.\\Test\\Modified\\ISIC_0000014.jpg', '.\\Test\\Modified\\ISIC_0000018.jpg', '.\\Test\\Modified\\ISIC_0000022.jpg']


In [24]:
leftout_test=[]
segtest_names = []
for i in range(len(segtest_ims)):
    name = segtest_ims[i]
    name = name.replace('.\\Test\\Modified\\','')
    name = name.replace('_seg.png','')
    segtest_names.append(name)
test_names = []
for i in range(len(test_ims)):
    name = test_ims[i]
    name = name.replace('.\\Test\\Modified\\','')
    name = name.replace('.jpg','')
    test_names.append(name)
print(segtest_names[:5])
print(test_names[:5])
leftout_test = [x for x in test_names if x not in segtest_names]
print(len(leftout_test))
print(leftout_test[:5])

['ISIC_0000006', 'ISIC_0000011', 'ISIC_0000014', 'ISIC_0000018', 'ISIC_0000022']
['ISIC_0000006', 'ISIC_0000011', 'ISIC_0000014', 'ISIC_0000018', 'ISIC_0000022']
25
['ISIC_0012459', 'ISIC_0014179', 'ISIC_0014636', 'ISIC_0024787', 'ISIC_0026567']


In [25]:
leftout_seg = []
for i in range(len(leftout_test)):
    name = leftout_test[i]
    name = './Test/Modified/'+name+'_seg.png'
    leftout_seg.append(name)
print(leftout_seg[:5])

['./Test/Modified/ISIC_0012459_seg.png', './Test/Modified/ISIC_0014179_seg.png', './Test/Modified/ISIC_0014636_seg.png', './Test/Modified/ISIC_0024787_seg.png', './Test/Modified/ISIC_0026567_seg.png']


In [26]:
for i in range(len(leftout_seg)):
    im = np.zeros((384,512))
    cv.imwrite(leftout_seg[i],im)

In [28]:
segtrain_ims = sorted(glob(os.path.join('.\Train\Modified','*.png')))
print(len(segtrain_ims))
print(segtrain_ims[:5])
train_ims = sorted(glob(os.path.join('.\Train\Modified','*.jpg')))
print(len(train_ims))
print(train_ims[:5])

18989
['.\\Train\\Modified\\ISIC_0000000_seg.png', '.\\Train\\Modified\\ISIC_0000001_seg.png', '.\\Train\\Modified\\ISIC_0000002_seg.png', '.\\Train\\Modified\\ISIC_0000003_seg.png', '.\\Train\\Modified\\ISIC_0000004_seg.png']
18998
['.\\Train\\Modified\\ISIC_0000000.jpg', '.\\Train\\Modified\\ISIC_0000001.jpg', '.\\Train\\Modified\\ISIC_0000002.jpg', '.\\Train\\Modified\\ISIC_0000003.jpg', '.\\Train\\Modified\\ISIC_0000004.jpg']


In [29]:
leftout_train=[]
segtrain_names = []
for i in range(len(segtrain_ims)):
    name = segtrain_ims[i]
    name = name.replace('.\\Train\\Modified\\','')
    name = name.replace('_seg.png','')
    segtrain_names.append(name)
train_names = []
for i in range(len(train_ims)):
    name = train_ims[i]
    name = name.replace('.\\Train\\Modified\\','')
    name = name.replace('.jpg','')
    train_names.append(name)
print(segtrain_names[:5])
print(train_names[:5])
leftout_train = [x for x in train_names if x not in segtrain_names]
print(len(leftout_train))
print(leftout_train[:5])

['ISIC_0000000', 'ISIC_0000001', 'ISIC_0000002', 'ISIC_0000003', 'ISIC_0000004']
['ISIC_0000000', 'ISIC_0000001', 'ISIC_0000002', 'ISIC_0000003', 'ISIC_0000004']
9
['ISIC_0012254', 'ISIC_0025831', 'ISIC_0031814', 'ISIC_0031937', 'ISIC_0057964']


In [30]:
leftout_seg = []
for i in range(len(leftout_train)):
    name = leftout_train[i]
    name = './Train/Modified/'+name+'_seg.png'
    leftout_seg.append(name)
print(leftout_seg[:5])

['./Train/Modified/ISIC_0012254_seg.png', './Train/Modified/ISIC_0025831_seg.png', './Train/Modified/ISIC_0031814_seg.png', './Train/Modified/ISIC_0031937_seg.png', './Train/Modified/ISIC_0057964_seg.png']


In [31]:
for i in range(len(leftout_seg)):
    im = np.zeros((384,512))
    cv.imwrite(leftout_seg[i],im)